In [ ]:
import os, sys
sys.path.insert(0, os.getcwd()+ '/..') 

import pandas as pd
import json
from collections import namedtuple

from gather_datasets.utils import get_files
from sys import getsizeof

from tqdm import tqdm

In [ ]:
ratings_df = pd.read_csv(f'../datasets/spotify/ratings.csv.gz')
ratings_df

In [ ]:
tracks_df = pd.read_csv(f'../datasets/spotify/tracks.csv.gz')

In [ ]:
pd.set_option('display.width', 300)
tracks_df

In [ ]:
TrackInfo = namedtuple('TrackInfo', ['track_id', 'track_uri', 'track_name', 'artist_uri', 'artist_name', 'album_uri'])

tracks = {}
playlists = []
additional_playlist_info = []

next_id = 0

for file in tqdm(get_files('../datasets/downloads/spotify/data')):
    with open(file, 'r') as f:
        data = json.load(f)
        playlists_data = data['playlists']
        for playlist in playlists_data:
            
            pid = playlist['pid']
            timestamp = playlist['modified_at']
            collaborative = playlist['collaborative'] == 'true'
            additional_playlist_info.append((pid, timestamp, collaborative))

            tracks_data = playlist['tracks']
            for track in tracks_data:
                track_uri = track['track_uri']
                
                id = None
                if track_uri in tracks:
                    id = tracks[track_uri].track_id
                else:
                    id = next_id
                    next_id += 1
                    tracks[track_uri] = TrackInfo(id, track_uri, track['track_name'], track['artist_uri'], track['artist_name'], track['album_uri'])
                playlists.append((pid, id))

In [ ]:
playlists_df = pd.DataFrame.from_records(additional_playlist_info, columns=['playlist_id', 'timestamp', 'collaborative'])

ratings_df = pd.DataFrame.from_records(playlists, columns=['playlist_id', 'track_id'])

tracks_df = pd.DataFrame.from_records(list(tracks.values()), columns=TrackInfo._fields)
tracks_df.reset_index(inplace=True, drop=True)
tracks_df.set_index('track_id', inplace=True)


In [ ]:
playlists_df.info()

In [ ]:
value_counts = playlists_df['collaborative'].value_counts()
print(f'Collaborative playlists ratio percent: {value_counts[True] / value_counts[False] * 100}%')

In [ ]:
print(f'Number of ratings: {len(ratings_df)}')

In [ ]:
print(f'Number of items: {len(tracks)}')

In [ ]:
# size = getsizeof(tracks)
# size += sum(map(getsizeof, tracks.values())) + sum(map(getsizeof, tracks.keys()))
# print(f'{size/1024/1024} MB')


In [ ]:
# size = getsizeof(playlists)
# print(f'{size/1024/1024} MB')

In [ ]:
# print(ratings_df.info())
# print(tracks_df.info())

In [ ]:
# tracks_df

In [ ]:
# ratings_df

In [ ]:
# ratings_df['pid'].nunique()